In [ ]:
## Code to generate all k-vertex-critical (P5,gem)-free graphs 
## Author: Ben Cameron, The King's University, November 3, 2022
## Based on work with Chính Hoàng
## Written in version 9.1 of sage. 

In [1]:
# get the 10 important (P5,gem)-free graphs with labels and edges according to:
#
# [M. Chudnovsky, T. Karthick, P. Maceli, and F. Maffray. 
#Coloring graphs with no induced five-vertex path or gem. 
#Journal of Graph Theory, 95(4):527–542, 2020.]

G1 = Graph([{1,2},{2,3},{3,4},{4,5},{5,1}])
G2 = Graph([{1,2},{2,3},{3,4},{4,5},{5,1},{6,1},{6,3},{6,4}])
G3 = Graph([{1,2},{2,3},{3,4},{4,5},{5,1},{6,1},{6,3},{6,4},{7,4},{7,2}])
G4 = Graph([{1,2},{2,3},{3,4},{4,5},{5,1},{6,1},{6,3},{6,4},{7,4},{7,1},{7,2}])
G5 = Graph([{1,2},{2,3},{3,4},{4,5},{5,1},{6,1},{6,3},{6,4},{7,4},{7,2},{7,1},{8,1},{8,3}])
G6 = Graph([{1,2},{2,3},{3,4},{4,5},{5,1},{6,1},{6,3},{6,4},{6,8},{7,1},{7,2},{7,4},{8,2},{8,5}])
G7 = Graph([{1,2},{1,3},{1,4},{1,7},{2,3},{2,5},{2,8},{3,6},{4,5},{4,6},{4,8},{5,6},{5,7}])
G8 = Graph([{1,2},{2,3},{3,4},{4,5},{5,1},{6,1},{6,4},{6,8},{7,8},{7,3},{7,1},{7,5},{8,4},{8,2}])
G9 = Graph([{1,2},{2,3},{3,4},{4,5},{5,1},{6,1},{6,4},{6,8},{7,8},{7,3},{7,1},{7,5},{8,4},{8,2},{9,1},{9,4}])
G10= Graph([{1,2},{2,3},{3,4},{4,5},{5,1},{6,1},{6,4},{6,8},{6,9},{7,8},{7,3},{7,1},{7,5},{8,4},{8,2},{9,2},{9,3},{9,5}])
Glist = [G1,G2,G3,G4,G5,G6,G7,G8,G9,G10]

In [2]:
#we will need to compute various clique expansions of the graphs in Glist
#The following function will be used for this:

#
# returns the clique expansion of the graph G on vertex set [1,...,n] by replacing vertex V[i] of G with a clique 
# of order L[i] (len(L) must equal order of G and each entry a positive integer)
#
def clique_expansion(G,L): 
    Gverts = G.vertices()
    n = len(Gverts)
    if len(L) != n:
        return("Error: list not the same size as order of G")
    Gedges = G.edges()
    V = [[] for i in range(n)] #vertex set of final graph partitioned into each clique
    E = [] #edge set of final graph
    for i in range(n): # loop fills out
        for j in range(L[i]):
            V[i].append((i+1,j+1))
    for i in range(n): #adds all edges in the cliques
        C = Combinations(V[i], 2).list()
        for c in C:
            E.append({c[0],c[1]})
    for e in Gedges: #adds edges between cliques based on adjaceny of G
        x = e[0]-1
        y = e[1]-1
        for v in V[x]:
            for u in V[y]:
                E.append({u,v})
    return(Graph(E)) 

In [3]:
# Since the k-vertex-critical test is computationally expensive,  we reduce the search space and we will only test clique nonisomorhpic clique expansions
# the following function will be used to generate all possible clique expansions
# it also takes into account those whose minimum degree are not high enough or whose clique number is already too large to be k-colourable

#
# function gives list of possible n-tuples (n is order of G) that can be used
# for clique expansions of G and still have clique number < k
# and further do not result in isomorphic clique expansions
# Graph must have vertices labeled 1,...,n
#
def possible_kcrit_tuples(G,k):
    V = G.vertices()
    n = len(V)
    T = Tuples([i for i in range(1,k-1)],n).list() #list of all n-tuples to be pruned to valid ones only can only do cliques up to order k-2, otherwise, since no isolated verts, clique of size k immediately 
    E = G.edges(labels = False)
    C = list(sage.graphs.cliquer.all_cliques(G, min_size=2))
    #print(C)
    for i in range(len(C)):
        c = C[i]
        badTups = []
        for t in T:
            sm = 0
            for m in c:
                sm += t[m-1]
                if sm >= k:
                    badTups.append(t)#creates list of bad tuples for removal after loop
                    break
        for b in badTups:
            T.remove(b)
    #
    # At this point only tuples which do not put clique number too high remain
    # What follows is removing autmorphisms
    #
    badTups = []
    Gr = G.automorphism_group()
    ordG = Gr.order()
    for j in range(len(T)):
        C1 = T[j]
        for h in range(j+1,len(T)):
            C2 = T[h]
            if (C2 in badTups) == False:
                for i in range(ordG):
                    p = Permutation(Gr[i])
                    flag = True #assume isomorphic, flag otherwise
                    for o in range(n):
                        if C1[o] != C2[p[o]-1]:
                            flag = False
                            break
                    if flag:
                        badTups.append(C2)
                        break
    for l in badTups:
        T.remove(l)
    return(T)

In [4]:
#Brute-force test determines if given graph G is k-vertex-critical
# requires import of vertex_coloring for chromatic_number() and vertex_colouring()

from sage.graphs.graph_coloring import vertex_coloring

def is_kcrit(G,k): 
    chi = G.chromatic_number()
    if chi != k:
        return(False)
    V = G.vertices()
    n = len(V)
    vlist = Combinations(V,n-1).list()
    for l in vlist:
        H = G.subgraph(l,inplace=False)
        if vertex_coloring(H, k-1, value_only=True) == False:
            return False
    return True

In [5]:
#generate k=4
for i in range(len(Glist)):
    g= Glist[i]
    L = possible_kcrit_tuples(g,4)
    for l in L:
        H = clique_expansion(g,l)
        if is_kcrit(H,4):
            print(f"G{i+1}("+ ",".join(str(num) for num in l) + ")")
            print(H.graph6_string())


G1(2,1,2,1,1)
FxK]G
G7(1,1,1,1,1,1,2,2)
I{S{STOSG


In [6]:
#generate k=5
for i in range(len(Glist)):
    g= Glist[i]
    L = possible_kcrit_tuples(g,5)
    for l in L:
        H = clique_expansion(g,l)
        if is_kcrit(H,5):
            print(f"G{i+1}("+ ",".join(str(num) for num in l) + ")")
            print(H.graph6_string())


G1(3,1,3,1,1)
H~CWw^`
G1(3,1,2,2,1)
H~CWW^b
G1(2,2,2,2,1)
H~KwW^B
G7(1,2,1,2,1,1,3,2)
L~aJW}A_kDl_Z@
G7(1,1,2,2,1,1,3,2)
L~aIX}A_kDh_R@
G7(1,1,2,1,1,1,3,3)
L~`H{ID`YOH@QB


In [7]:
#generate k=6
for i in range(len(Glist)):
    g= Glist[i]
    L = possible_kcrit_tuples(g,6)
    for l in L:
        H = clique_expansion(g,l)
        if is_kcrit(H,6):
            print(f"G{i+1}("+ ",".join(str(num) for num in l) + ")")
            print(H.graph6_string())


G1(4,1,4,1,1)
J~{GW[N@~__
G1(4,1,3,2,1)
J~{GW[F@~`_
G1(3,2,3,2,1)
J~{Ww{F@~@_
G1(3,2,2,3,1)
J~{WwKF@~B_
G1(3,2,2,2,2)
J~{WwKFw^B_
G7(1,3,1,3,1,1,4,2)
O~}CKLvB{@OD_J_JmwBm@
G7(1,2,2,3,1,1,4,2)
O~}CKLfF{@OD_J_JkwBM@
G7(1,1,3,3,1,1,4,2)
O~}CKLFN{@OD_J_JgwAM@
G7(1,2,2,2,1,2,4,2)
O~}CJK^F{AOH_R_RkoBK@
G7(2,2,1,2,2,1,3,3)
O~~EHkvB}BWLoZL_EoHkB
G7(2,1,2,2,2,1,3,3)
O~~EHKfF}BWLoZH_CoHKB
G7(1,1,3,2,2,1,3,3)
O~}CILFN{BOL_ZP_GoIKB
G7(2,1,2,2,1,2,3,3)
O~~EHK^F}AWHoRH_CoHKB
G7(1,2,2,2,1,1,4,3)
O~}CJK^_SAoJ_VX_KoJKB
G7(1,1,3,2,1,1,4,3)
O~}CIK~_SAoJ_VP_GoIKB
G7(1,2,2,1,1,2,4,3)
O~}BG{^_cCoR_fX?K_JGB
G7(1,1,3,1,1,1,4,4)
O~}AH}A_kDoVP?P?g_YGF
G7(1,1,2,1,1,2,4,4)
O~`Hx}C`KHofQ?Q?h?YOF


In [8]:
#generate k=7
for i in range(len(Glist)):
    g= Glist[i]
    L = possible_kcrit_tuples(g,7)
    for l in L:
        H = clique_expansion(g,l)
        if is_kcrit(H,7):
            print(f"G{i+1}("+ ",".join(str(num) for num in l) + ")")
            print(H.graph6_string())


G1(5,1,5,1,1)
L~~wGKF@wN_^}@
G1(5,1,4,2,1)
L~~wGKF@wF_^}B
G1(4,2,4,2,1)
L~~wW[NBwF_^{B
G1(5,1,3,3,1)
L~~wGKF?wF_^}F
G1(4,2,3,3,1)
L~~wW[N?wF_^{F
G1(3,3,3,3,1)
L~~ww{^?wF_^wF
G1(4,2,2,4,1)
L~~wW[B?wF_^{N
G1(4,2,3,2,2)
L~~wW[N?wF}B{F
G1(3,3,3,2,2)
L~~ww{^?wF{BwF
G1(3,3,2,3,2)
L~~ww{B?wF{FwN
G7(1,4,1,4,1,1,5,2)
R~~{CEB_zv_~_A_Ao@[?V_A|zoBv_G
G7(1,3,2,4,1,1,5,2)
R~~{CEB_zf`~_A_Ao@[?V_A|roBf_G
G7(1,2,3,4,1,1,5,2)
R~~{CEB_zFb~_A_Ao@[?V_A|boBF_G
G7(1,1,4,4,1,1,5,2)
R~~{CEB_yFf~_A_Ao@[?V_A|BoAF_G
G7(1,3,2,3,1,2,5,2)
R~~{CEB[w^`~_C_CoA[?f_C|r_Bf?G
G7(1,2,3,3,1,2,5,2)
R~~{CEBWw~b~_C_CoA[?f_C|b_BF?G
G7(2,3,1,3,2,1,4,3)
R~~}EFBMxv_~oEoEwB]?vMw?z_Hv?W
G7(2,2,2,3,2,1,4,3)
R~~}EFBKxf`~oEoEwB]?vKw?r_Hf?W
G7(1,3,2,3,2,1,4,3)
R~~{CEB[zf`~_E_EoB[?v[w@r_Jf?W
G7(2,1,3,3,2,1,4,3)
R~~}EFBGxFb~oEoEwB]?vGw?b_HF?W
G7(1,2,3,3,2,1,4,3)
R~~{CEBWzFb~_E_EoB[?vWw@b_JF?W
G7(1,1,4,3,2,1,4,3)
R~~{CEBOyFf~_E_EoB[?vOw@B_IF?W
G7(2,2,2,3,1,2,4,3)
R~~}EFBKw^`~oCoCwA]?fKw?r_Hf?W
G7(2,1,3,3,1,2,4,3)
R~~}EFBGw~b~oCoCwA]?fGw?b_H